In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
from pymorphy2 import MorphAnalyzer
import sqlite3

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
morph = MorphAnalyzer()

In [4]:
def pos(word):
    return morph.parse(word)[0].tag.POS

In [5]:
def lemma(word):
  return morph.parse(word)[0].normal_form

In [6]:
conn = sqlite3.connect('poems_corpus.db')
cur = conn.cursor()

In [7]:
cur.execute("""
CREATE TABLE IF NOT EXISTS tokens (
    token_id INTEGER PRIMARY KEY AUTOINCREMENT,
    token TEXT,
    lemma TEXT,
    pos TEXT,
    sent_id int
)
""")

In [8]:
cur.execute('SELECT id_sent, sent FROM sentences')
for idx, sent in cur.fetchall():
  tokens = word_tokenize(sent)
  for token in tokens:
    token_lemma = lemma(token)
    token_pos = pos(token)
    cur.execute(
        """
        INSERT INTO tokens 
        (token, lemma, pos, sent_id) 
        VALUES (?, ?, ?, ?)
        """, (token, token_lemma, token_pos, idx)
    )

In [9]:
conn.commit()